In [1]:
import requests
import json
import pandas as pd
import numpy as np

# Landkreisdaten

In [2]:
LK_data = pd.read_csv('LK_data_clean.csv')
#source: https://de.wikipedia.org/wiki/Liste_der_Landkreise_in_Deutschland

In [3]:
def get_city_opendata(city, country):
    tmp = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldcitiespop&q=%s&sort=population&facet=country&refine.country=%s'
    cmd = tmp % (city, country)
    res = requests.get(cmd)
    dct = json.loads(res.content)
    out = dct['records'][0]['fields']
    return out

In [4]:
kreisstaedte = np.array(LK_data['Kreissitz'])

In [7]:
def get_kreisstadt_info(kreisstaedte):
    for idx,stadt in enumerate(kreisstaedte):
        if idx % 10 == 0:
            print(idx)
        try:
            dpt = get_city_opendata(stadt, 'de')
            data_lk.append(dpt)
        except:
            print('Stadt ', stadt, 'nicht im Datensatz')
    return data_lk

In [8]:
data_lk = []
data_lk = get_kreisstadt_info(kreisstaedte)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280


In [9]:
Einw_pro_LK = np.array(LK_data['Einw.'])
BL_pro_LK = np.array(LK_data['BL'])

In [11]:
#create dictionary
kreisstadt_dict = {
    f"kreisstadt{index}": {
        "city": data_lk[index]['city'],
        "ident": index,
        "position": {"lat": data_lk[index]['latitude'], "lon": data_lk[index]['longitude']},
        "population_LK": Einw_pro_LK[index],
        "BL": BL_pro_LK[index]
    }
    for index, elm in enumerate(data_lk)
} 

In [12]:
#check population accounted for
pop_cnt = 0
for i in range(290):
    pop_cnt += kreisstadt_dict[f"kreisstadt{i}"]['population_LK']
print(pop_cnt)

49343348


In [ ]:
# dump data
import csv

w = csv.writer(open("kreisstaedte_data_final.csv", "w"))
for key, val in kreisstadt_dict.items():
    w.writerow([key, val])

# Städtedaten

In [13]:
staedte_data = pd.read_csv('staedte_data.csv')
#source: https://de.wikipedia.org/wiki/Liste_der_kreisfreien_St%C3%A4dte_in_Deutschland

In [15]:
staedte = np.array(staedte_data['Stadt'])

In [17]:
def get_stadt_info(staedte):
    for idx,stadt in enumerate(staedte):
        if idx % 10 == 0:
            print(idx)
        try:
            dpt = get_city_opendata(stadt, 'de')
            data_stadt.append(dpt)
        except:
            print('Stadt ', stadt, 'nicht im Datensatz')
    return data_stadt

In [18]:
data_stadt = []
data_stadt = get_stadt_info(staedte)

0
10
20
30
40
50
60
70
80
90
100


In [19]:
print(type(data_stadt))

for i, elm in enumerate(data_stadt):
    try:
        data_stadt[i]['population']
    except:
        print(i, elm)
        print("Doesn't have population info")

<class 'list'>
13 {'city': 'barum braunschweig', 'country': 'de', 'region': '06', 'geopoint': [52.116667, 10.433333], 'longitude': 10.433333, 'latitude': 52.116667, 'accentcity': 'Barum Braunschweig'}
Doesn't have population info
33 {'city': 'frankfurt am main', 'country': 'de', 'region': '05', 'geopoint': [50.116667, 8.683333], 'longitude': 8.683333, 'latitude': 50.116667, 'accentcity': 'Frankfurt am Main'}
Doesn't have population info
57 {'city': 'koln', 'country': 'de', 'region': '07', 'geopoint': [50.933333, 6.95], 'longitude': 6.95, 'latitude': 50.933333, 'accentcity': 'Köln'}
Doesn't have population info
70 {'city': 'mulheim an der ruhr', 'country': 'de', 'region': '07', 'geopoint': [51.433333, 6.883333], 'longitude': 6.883333, 'latitude': 51.433333, 'accentcity': 'Mülheim an der Ruhr'}
Doesn't have population info
71 {'city': 'munchenhof', 'country': 'de', 'region': '14', 'geopoint': [51.833333, 11.116667], 'longitude': 11.116667, 'latitude': 51.833333, 'accentcity': 'Münchenhof

In [21]:
with open('stadt.txt') as f:
    lines = f.read().splitlines()

In [22]:
# rest is manual work